In [7]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from keras.layers import *
from keras.models import *
from keras.utils import *
from keras.initializers import *
from keras.optimizers import * 

In [12]:
batch_size = 64  
epochs = 20  
latent_dim = 256  
num_samples = 10000  

# set the data_path accordingly
data_path = r"E:\Pycharm_Projects\tf_prac\Encoder,Decoder,Seq2Seq\eng-spa\spa.txt"

In [3]:
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")
    
    
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text, _ = line.split("\t")
    ############### A ###############
    target_text = "\t" + target_text + "\n"
    input_texts.append(input_text)
    target_texts.append(target_text)
    ############### B ###############
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)
print(input_characters)
print(target_characters)

{'a', 'd', '.', 'F', 'f', 'N', '7', '5', '-', 'k', 'q', 'e', 'l', "'", 'D', '9', 'r', 'C', '6', 'b', 'c', 'T', 'U', 'h', 'g', 'u', 'J', 'z', '3', '0', 'S', 'Y', 'E', ',', 'm', '!', 'I', 'O', 'P', '$', 'j', ':', '2', 'x', 'n', 'K', 'w', 'M', 'Q', '?', 'p', 'V', '1', 'H', ' ', 'y', 'L', '8', 'i', 'R', 'W', '4', 'B', 's', 'G', 't', 'A', 'v', 'o'}
{'«', 'F', 'ü', 'k', 'C', 'g', '!', 'O', 'w', 'M', 'Q', 'V', 'p', 'É', ' ', 'i', 'Ú', 'W', '4', 't', 'A', 'v', 'o', 'é', 'ó', 'N', 'e', 'D', 'r', 'U', 'T', 'u', '3', '0', 'S', 'Y', 'E', '"', 'm', 'P', 'j', 'Ó', '\t', '2', 'x', 'n', '?', 'H', 'R', 'G', '.', 'f', '7', '5', '¡', 'q', 'l', "'", '6', 'ñ', 'h', '¿', 'z', ',', '1', '»', '8', 'B', 's', 'a', 'd', '-', 'c', 'J', 'í', 'I', '\n', ':', 'Á', 'ú', 'L', 'y', 'á', 'b'}


In [4]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])
print("No.of samples:", len(input_texts))
print("No.of unique input tokens:", num_encoder_tokens)
print("No.of unique output tokens:", num_decoder_tokens)
print("Maximum seq length for inputs:", max_encoder_seq_length)
print("Maximum seq length for outputs:", max_decoder_seq_length)

No.of samples: 10000
No.of unique input tokens: 69
No.of unique output tokens: 84
Maximum seq length for inputs: 16
Maximum seq length for outputs: 47


In [5]:
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])
print(input_token_index)
print(target_token_index)

{' ': 0, '!': 1, '$': 2, "'": 3, ',': 4, '-': 5, '.': 6, '0': 7, '1': 8, '2': 9, '3': 10, '4': 11, '5': 12, '6': 13, '7': 14, '8': 15, '9': 16, ':': 17, '?': 18, 'A': 19, 'B': 20, 'C': 21, 'D': 22, 'E': 23, 'F': 24, 'G': 25, 'H': 26, 'I': 27, 'J': 28, 'K': 29, 'L': 30, 'M': 31, 'N': 32, 'O': 33, 'P': 34, 'Q': 35, 'R': 36, 'S': 37, 'T': 38, 'U': 39, 'V': 40, 'W': 41, 'Y': 42, 'a': 43, 'b': 44, 'c': 45, 'd': 46, 'e': 47, 'f': 48, 'g': 49, 'h': 50, 'i': 51, 'j': 52, 'k': 53, 'l': 54, 'm': 55, 'n': 56, 'o': 57, 'p': 58, 'q': 59, 'r': 60, 's': 61, 't': 62, 'u': 63, 'v': 64, 'w': 65, 'x': 66, 'y': 67, 'z': 68}
{'\t': 0, '\n': 1, ' ': 2, '!': 3, '"': 4, "'": 5, ',': 6, '-': 7, '.': 8, '0': 9, '1': 10, '2': 11, '3': 12, '4': 13, '5': 14, '6': 15, '7': 16, '8': 17, ':': 18, '?': 19, 'A': 20, 'B': 21, 'C': 22, 'D': 23, 'E': 24, 'F': 25, 'G': 26, 'H': 27, 'I': 28, 'J': 29, 'L': 30, 'M': 31, 'N': 32, 'O': 33, 'P': 34, 'Q': 35, 'R': 36, 'S': 37, 'T': 38, 'U': 39, 'V': 40, 'W': 41, 'Y': 42, 'a': 43,

In [8]:
encoder_input_data = np.zeros(
  (len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32"
)

decoder_input_data = np.zeros(
  (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)
decoder_target_data = np.zeros(
  (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0
    for t, char in enumerate(target_text):
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
    decoder_target_data[i, t:, target_token_index[" "]] = 1.0

In [9]:
encoder_inputs = keras.Input(shape=(None, num_encoder_tokens))
encoder = keras.layers.LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

In [10]:
decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))
decoder_lstm = keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

In [13]:
model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.compile(optimizer=Adam(lr=0.01, beta_1=0.9, beta_2=0.999, decay=0.001), loss='categorical_crossentropy', metrics=["accuracy"])

model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2,
)
model.save("E2S")

Epoch 1/20
125/125 [==============================] - 51s 375ms/step - loss: 0.4675 - accuracy: 0.8589 - val_loss: 0.5686 - val_accuracy: 0.8301
Epoch 2/20
125/125 [==============================] - 42s 334ms/step - loss: 0.4211 - accuracy: 0.8724 - val_loss: 0.5527 - val_accuracy: 0.8351
Epoch 3/20
125/125 [==============================] - 43s 344ms/step - loss: 0.3938 - accuracy: 0.8805 - val_loss: 0.5350 - val_accuracy: 0.8405
Epoch 4/20
125/125 [==============================] - 44s 350ms/step - loss: 0.3667 - accuracy: 0.8887 - val_loss: 0.5281 - val_accuracy: 0.8431
Epoch 5/20
125/125 [==============================] - 40s 316ms/step - loss: 0.3438 - accuracy: 0.8954 - val_loss: 0.5212 - val_accuracy: 0.8473
Epoch 6/20
125/125 [==============================] - 45s 364ms/step - loss: 0.3206 - accuracy: 0.9024 - val_loss: 0.5142 - val_accuracy: 0.8513
Epoch 7/20
125/125 [==============================] - 47s 375ms/step - loss: 0.2994 - accuracy: 0.9086 - val_loss: 0.5149 - val_ac

INFO:tensorflow:Assets written to: E2S\assets


INFO:tensorflow:Assets written to: E2S\assets


In [ ]:
from keras.utils import plot_model

plot_model(model, to_file='modelsummary.png', show_shapes=True, show_layer_names=True)

In [14]:
print("shape encoder_input_data :",encoder_input_data.shape)
print("shape decoder_input_data :",decoder_input_data.shape)
print("shape decoder_target_data:",decoder_target_data.shape)

shape encoder_input_data : (10000, 16, 69)
shape decoder_input_data : (10000, 47, 84)
shape decoder_target_data: (10000, 47, 84)


In [15]:
encoder_inputs = model.input[0]  # input_1
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output  # lstm_1
encoder_states = [state_h_enc, state_c_enc]
encoder_model = keras.Model(encoder_inputs, encoder_states)

In [16]:
decoder_inputs = model.input[1]  # input_2
decoder_state_input_h = keras.Input(shape=(latent_dim,), name="input_3")
decoder_state_input_c = keras.Input(shape=(latent_dim,), name="input_4")
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = model.layers[3]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs
)
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = keras.Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)

In [17]:
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

In [18]:
def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, target_token_index["\t"]] = 1.0

    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        states_value = [h, c]
    return decoded_sentence

In [19]:
i = np.random.choice(len(input_texts))
input_seq = encoder_input_data[i:i+1]
translation = decode_sequence(input_seq)
print('-')
print('Input:', input_texts[i])
print('Translation:', translation)

1/1 [==============================] - 0s 31ms/step
-
Input: We know it now.
Translation: Lo sabemos.

